In [3]:
import numpy as np
import pandas as pd
import requests

import sys

sys.path.append("../")

from mods import EPA
from mods import EIA


In [56]:
eia = pd.read_csv("./EIA_Plant_List.csv")
epa = pd.read_csv("./EPA_plant_List.csv")
walk = pd.read_csv("./TX_crosswalk.csv")

### EIA Matches

In [32]:
print("Number of EIA Matches: ", len(eia.loc[eia["match"] == True]))
print("Number of EIA Non-Matches: ", len(eia.loc[eia["match"] == False]))

print("Percent of EIA Matches: ", round(len(eia.loc[eia["match"] == True])/len(eia), 2))
print("Percent of EIA Non-Matches: ", round(len(eia.loc[eia["match"] == False])/len(eia), 2))

Number of EIA Matches:  429
Number of EIA Non-Matches:  707
Percent of EIA Matches:  0.38
Number of EIA Non-Matches:  0.62


### EPA Matches

In [33]:
print("Number of EPA Matches: ", len(epa.loc[epa["match"] == True]))
print("Number of EPA Non-Matches: ", len(epa.loc[epa["match"] == False]))

print("Percent of EPA Matches: ", round(len(epa.loc[epa["match"] == True])/len(eia), 2))
print("Percent of EPA Non-Matches: ", round(len(epa.loc[epa["match"] == False])/len(eia), 2))

Number of EIA Matches:  414
Number of EIA Non-Matches:  100
Percent of EIA Matches:  0.36
Number of EIA Non-Matches:  0.09


### EIA Fuel Breakdown

In [29]:
print("\nPercent Fuel Types Matched\n")
print(eia.loc[eia["match"] == True]["Energy Source 1"].value_counts(normalize=True))
print("\nPercent Fuel Types Not Matched\n")
print(eia.loc[eia["match"] == False]["Energy Source 1"].value_counts(normalize=True))


Percent Fuel Types Matched

Natural Gas    0.927739
Coal           0.072261
Name: Energy Source 1, dtype: float64

Percent Fuel Types Not Matched

Natural Gas    0.371994
Wind           0.281471
Solar          0.114569
Renewable      0.072136
Hydro          0.070721
Petroleum      0.042433
Other          0.041018
Nuclear        0.005658
Name: Energy Source 1, dtype: float64


### EPA Fuel Breakdown

In [57]:
print("\nPercent Fuel Types Matched\n")
print(epa.loc[epa["match"] == True]["fuel"].value_counts(normalize=True))
print("\nPercent Fuel Types Not Matched\n")
print(epa.loc[epa["match"] == False]["fuel"].value_counts(normalize=True))


Percent Fuel Types Matched

Pipeline Natural Gas    0.874396
Coal                    0.101449
Natural Gas             0.012077
NONE                    0.007246
Other Gas               0.004831
Name: fuel, dtype: float64

Percent Fuel Types Not Matched

Pipeline Natural Gas    0.92
NONE                    0.06
Petroleum Coke          0.01
Wood                    0.01
Name: fuel, dtype: float64


### Aggregate EPA vs EIA MWh in Q1 2019

In [6]:
epa_api = EPA("9ndbfvcHxIEgQ8KnDGhmVFdw3xiyOgqhhwdJg5Wo")
eia_api = EIA("9ndbfvcHxIEgQ8KnDGhmVFdw3xiyOgqhhwdJg5Wo")

epa_data = epa_api.hourlyData(9, "CTG-1", "2019", 1)
eia_data = eia_api.electric_plant_all(9)

TypeError: 'module' object is not callable